In [1]:
## Basic imports 
import pandas as pd
import numpy as np
import io
import os
import tensorflow as tf
from tensorflow import keras
from keras import layers
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [2]:
import keras 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'gender_images/'

os.chdir(base_dir)
os.listdir()

path_training = 'Training/'
path_validation = 'Validation/'

pic_path = path_training+'female/131435.jpg.jpg'
image_size = (90, 90)
batch_size = 64

target_size = (224, 224)
input_shape=(224, 224, 3)
min_lr = 0.0001
steps_per_epoch=256
validation_steps=256
epochs=8


train_datagen = ImageDataGenerator(rescale = 1./255,
      rotation_range=25,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')








test_datagen = ImageDataGenerator( rescale = 1.0/255)

train_generator = train_datagen.flow_from_directory(
    path_training,
    seed=10,
    target_size=(224,224),
    batch_size=batch_size,
    classes=['female','male'],
    shuffle=True,
)

validation_generator = test_datagen.flow_from_directory(
    path_validation,
    seed=10,
    target_size=(224,224),
    batch_size=batch_size,
    classes=['female','male'],
    shuffle=True
)




from matplotlib.image import imread

takeapeak = imread(pic_path)
takeapeak.shape







Found 47035 images belonging to 2 classes.
Found 11643 images belonging to 2 classes.


(98, 76, 3)

In [3]:
## Build the VGG model 

from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import Sequential,load_model,save_model
from tensorflow.keras.layers import Dense,Conv2D,Flatten,MaxPooling2D
from keras.layers import BatchNormalization
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Flatten, Dense
from keras.applications import VGG16



vgg_model = keras.applications.VGG16(
    weights='imagenet',  
    input_shape=(224, 224, 3),
    include_top=False)
vgg_model.trainable = False



inputs = keras.Input(shape=(224, 224, 3))
x = vgg_model(inputs, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)
epochs=20
model.compile(optimizer=keras.optimizers.Adam(1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()



58889256/58889256 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 1)                 513       
                                                                 
Total params: 14,715,201
Trainable params: 513
Non-trainable params: 14,714,688
_________________________________________________________________


In [ ]:

vgg16_fit = model.fit(train_generator, epochs=epochs, validation_data=validation_generator)




Epoch 1/20
 14/735 [..............................] - ETA: 6:52:53 - loss: 7.7125 - accuracy: 0.5000

In [ ]:

from tensorflow.keras.utils import plot_model
plot_model(model,show_shapes=True, show_layer_names=True, rankdir='TB', expand_nested=True)




Epoch 1/20
11/11 [==============================] - 450s 39s/step - loss: 0.9298 - accuracy: 0.5000 - val_loss: 0.7142 - val_accuracy: 0.5000
Epoch 2/20
 5/11 [============>.................] - ETA: 2:59 - loss: 0.7711 - accuracy: 0.5000

KeyboardInterrupt: ignored

In [ ]:
acc= vgg16_fit.history['accuracy']
val_acc = vgg16_fit.history['val_accuracy']

loss = vgg16_fit.history['loss']
val_loss = vgg16_fit.history['val_loss']

import matplotlib.pyplot as plt

epochs_range = range(epochs)

plt.figure(figsize=(8, 10))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()